# L'objectif de cet exercice est de réaliser un modèle de machine learning (IA) basé sur la régression logistique capable de prédire si un patient à une maladie cardiaque ou non. Pour cela, nous mettons à votre disposition un jeu de données de 918 patients.

# 1.	Charger le fichier de données Coeur.xlsx dans un objet data.

In [29]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
data = pd.read_excel('Coeur.xlsx')
data.head(10)

,AGE,SEXE,TDT,PAR,CHOLESTEROL,GAJ,ECG,FCMAX,ANGINE,DEPRESSION,PENTE,CŒUR
0,40,homme,AA,140,289,0,Normal,172,Non,0.0,Ascendant,0
1,49,femme,DNA,160,180,0,Normal,156,Non,1.0,Plat,1
2,37,homme,AA,130,283,0,ST,98,Non,0.0,Ascendant,0
3,48,femme,ASY,138,214,0,Normal,108,Oui,1.5,Plat,1
4,54,homme,DNA,150,195,0,Normal,122,Non,0.0,Ascendant,0
5,39,homme,DNA,120,339,0,Normal,170,Non,0.0,Ascendant,0
6,45,femme,AA,130,237,0,Normal,170,Non,0.0,Ascendant,0
7,54,homme,AA,110,208,0,Normal,142,Non,0.0,Ascendant,0
8,37,homme,ASY,140,207,0,Normal,130,Oui,1.5,Plat,1
9,48,femme,AA,120,284,0,Normal,120,Non,0.0,Ascendant,0


# 2.	Copiez l’objet data dans un nouvel objet df avec lequel vous allez travailler pour le reste de l’exercice.

In [3]:
df = data.copy()
df

,AGE,SEXE,TDT,PAR,CHOLESTEROL,GAJ,ECG,FCMAX,ANGINE,DEPRESSION,PENTE,CŒUR
0,40,homme,AA,140,289,0,Normal,172,Non,0.0,Ascendant,0
1,49,femme,DNA,160,180,0,Normal,156,Non,1.0,Plat,1
2,37,homme,AA,130,283,0,ST,98,Non,0.0,Ascendant,0
3,48,femme,ASY,138,214,0,Normal,108,Oui,1.5,Plat,1
4,54,homme,DNA,150,195,0,Normal,122,Non,0.0,Ascendant,0
...,...,...,...,...,...,...,...,...,...,...,...,...
913,45,homme,AT,110,264,0,Normal,132,Non,1.2,Plat,1
914,68,homme,ASY,144,193,1,Normal,141,Non,3.4,Plat,1
915,57,homme,ASY,130,131,0,Normal,115,Oui,1.2,Plat,1
916,57,femme,AA,130,236,0,LVH,174,Non,0.0,Plat,1


# 3.	Vérifiez s’il y a des doublons dans l’objet df ; si c’est le cas, supprimez-les.

In [4]:
#df.drop_duplicates(keep = 'first', inplace=True) on utilise pour supprimer au cas ou il y a des doublons
df.duplicated().sum()

0

# 4.	Vérifier s’il y a des constantes (variable avec une seule modalité) dans df ; si c’est le cas, supprimez les.

In [5]:
#df.nunique() sil y avait des constantes, on allait utiliser df = df.drop('...', axis=1) pour supprimer ...
df.nunique()

AGE             50
SEXE             2
TDT              4
PAR             67
CHOLESTEROL    222
GAJ              2
ECG              3
FCMAX          119
ANGINE           2
DEPRESSION      53
PENTE            3
CŒUR             2
dtype: int64

# 5.	Vérifier s’il y a des données manquantes dans df ; si c’est le cas, supprimez les.

In [6]:
# df = df.dropna(axis=0) on utilise pour supprimer au cas ou il y a des donnees manquantes
df.isna().sum()

AGE            0
SEXE           0
TDT            0
PAR            0
CHOLESTEROL    0
GAJ            0
ECG            0
FCMAX          0
ANGINE         0
DEPRESSION     0
PENTE          0
CŒUR           0
dtype: int64

# 6.	Faite une normalisation de toutes les variables quantitatives en divisant les valeurs de chaque colonne par la moyenne de celle-ci. Attention, la variable cible (COEUR) n'est pas à normaliser.

In [19]:
# on selectionne tout sauf object puis on enleve coeur
df_quant = df.select_dtypes(exclude='object').drop('CŒUR', axis=1)

In [20]:
df_quant = df.select_dtypes(exclude='object').drop('CŒUR', axis=1).columns
for col in df_quant:
    df[col]=pd.DataFrame(df[col]/df[col].mean())
df_norm = df[df_quant]
df_norm
#     df_quant = pd.DataFrame(df_quant[i]/df_quant[i].mean(), index = df_quant.index, columns = df_quant.columns)

,AGE,SEXE,TDT,PAR,CHOLESTEROL,GAJ,ECG,FCMAX,ANGINE,DEPRESSION,PENTE
0,0.747511,1.266207,0.000000,1.057430,1.453726,0.00000,1.011013,1.257224,0.000000,0.000000,0.000000
1,0.915701,0.000000,2.300752,1.208491,0.905435,0.00000,1.011013,1.140273,0.000000,1.126933,1.867752
2,0.691448,1.266207,0.000000,0.981899,1.423544,0.00000,2.022026,0.716325,0.000000,0.000000,0.000000
3,0.897014,0.000000,0.766917,1.042324,1.076461,0.00000,1.011013,0.789420,2.474394,1.690400,1.867752
4,1.009140,1.266207,2.300752,1.132960,0.980887,0.00000,1.011013,0.891752,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...
913,0.840950,1.266207,1.533835,0.830838,1.327971,0.00000,1.011013,0.964846,0.000000,1.352320,1.867752
914,1.270769,1.266207,0.766917,1.087642,0.970827,4.28972,1.011013,1.030631,0.000000,3.831574,1.867752
915,1.065204,1.266207,0.766917,0.981899,0.658955,0.00000,1.011013,0.840586,2.474394,1.352320,1.867752
916,1.065204,0.000000,0.000000,0.981899,1.187125,0.00000,0.000000,1.271843,0.000000,0.000000,1.867752


#  7.	Faite un recodage de toutes les variables qualitatives.

In [72]:
def recoder(data): 
    return data.astype('category').cat.codes

def recoder_df(data):
    for col in data.select_dtypes('object').columns:
        data[col] = recoder(data[col])

    return data

df = recoder_df(df)
df

,AGE,SEXE,TDT,PAR,CHOLESTEROL,GAJ,ECG,FCMAX,ANGINE,DEPRESSION,PENTE,CŒUR
0,0.747511,1.266207,0.000000,1.057430,1.453726,0.00000,1.011013,1.257224,0.000000,0.000000,0.000000,0
1,0.915701,0.000000,2.300752,1.208491,0.905435,0.00000,1.011013,1.140273,0.000000,1.126933,1.867752,1
2,0.691448,1.266207,0.000000,0.981899,1.423544,0.00000,2.022026,0.716325,0.000000,0.000000,0.000000,0
3,0.897014,0.000000,0.766917,1.042324,1.076461,0.00000,1.011013,0.789420,2.474394,1.690400,1.867752,1
4,1.009140,1.266207,2.300752,1.132960,0.980887,0.00000,1.011013,0.891752,0.000000,0.000000,0.000000,0
...,...,...,...,...,...,...,...,...,...,...,...,...
913,0.840950,1.266207,1.533835,0.830838,1.327971,0.00000,1.011013,0.964846,0.000000,1.352320,1.867752,1
914,1.270769,1.266207,0.766917,1.087642,0.970827,4.28972,1.011013,1.030631,0.000000,3.831574,1.867752,1
915,1.065204,1.266207,0.766917,0.981899,0.658955,0.00000,1.011013,0.840586,2.474394,1.352320,1.867752,1
916,1.065204,0.000000,0.000000,0.981899,1.187125,0.00000,0.000000,1.271843,0.000000,0.000000,1.867752,1


# 8.	On souhaite séparer la variable cible COEUR des autres variables de votre objet df. Pour cela, stocker la variable COEUR dans un objet appelé Y et les autres variables dans un objet appelé X.

In [34]:
y = df['CŒUR']
X = df.drop('CŒUR', axis=1)
X

,AGE,SEXE,TDT,PAR,CHOLESTEROL,GAJ,ECG,FCMAX,ANGINE,DEPRESSION,PENTE
0,0.747511,1.266207,0.000000,1.057430,1.453726,0.00000,1.011013,1.257224,0.000000,0.000000,0.000000
1,0.915701,0.000000,2.300752,1.208491,0.905435,0.00000,1.011013,1.140273,0.000000,1.126933,1.867752
2,0.691448,1.266207,0.000000,0.981899,1.423544,0.00000,2.022026,0.716325,0.000000,0.000000,0.000000
3,0.897014,0.000000,0.766917,1.042324,1.076461,0.00000,1.011013,0.789420,2.474394,1.690400,1.867752
4,1.009140,1.266207,2.300752,1.132960,0.980887,0.00000,1.011013,0.891752,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...
913,0.840950,1.266207,1.533835,0.830838,1.327971,0.00000,1.011013,0.964846,0.000000,1.352320,1.867752
914,1.270769,1.266207,0.766917,1.087642,0.970827,4.28972,1.011013,1.030631,0.000000,3.831574,1.867752
915,1.065204,1.266207,0.766917,0.981899,0.658955,0.00000,1.011013,0.840586,2.474394,1.352320,1.867752
916,1.065204,0.000000,0.000000,0.981899,1.187125,0.00000,0.000000,1.271843,0.000000,0.000000,1.867752


# 9.	En utilisant la fonction train_test_split du module model_selection du package sklearn, subdiviser vos données X et Y en respectivement  X_train, X_test et Y_train, Y_test. X_train et Y_train doivent contenir respectivement 80 % des données de X et Y.

In [13]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
#pour diviser le dataset, on aura 20% pour entrainer. random_state = 1 pour controler l'aleatoire

print('Train set:', X_train.shape)
print('Test set:', X_test.shape)

Train set: (734, 11)
Test set: (184, 11)


# 10.	En utilisant la classe LogisticRegression du module linear_model du package sklearn, construisez un modèle de régression logistique en utilisant les données X_train et Y_train et la méthode fit de la classe LogisticRegression.

In [69]:
from sklearn.linear_model import LogisticRegression
LogReg = LogisticRegression()
LogReg.fit(X_train, y_train)

LogisticRegression()

# 11.	En utilisant les données de X_test et la méthode predict_proba de la classe LogisticRegression affichez la probabilité d'appartenance à chacune des classes de la variable COEUR. 

In [70]:
LogReg.predict_proba(X_test[:10])

array([[0.97034204, 0.02965796],
       [0.91540722, 0.08459278],
       [0.93719591, 0.06280409],
       [0.05457318, 0.94542682],
       [0.04245397, 0.95754603],
       [0.83773383, 0.16226617],
       [0.44256046, 0.55743954],
       [0.92153361, 0.07846639],
       [0.17396695, 0.82603305],
       [0.92589969, 0.07410031]])

In [67]:
cardiologie(model)

[0.5444328]


# 12.	En utilisant les données de X_test et la méthode predict de la classe LogisticRegression prédisez la classe d'appartenance de chaque individu de X_test. 

In [71]:
LogReg.predict(X_test)

array([0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0,
       1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0,
       0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1,
       0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1,
       0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0,
       1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0,
       1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0,
       1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1,
       0, 0, 1, 1, 1, 0, 1, 0], dtype=int64)

# 13.	En utilisant les données de X_test et Y_test et les méthodes confusion_matrix, accuracy_score, recall_score, et precision_score, du module metrics du package sklearn, affichez respectivement la matrice de confusion, le taux succès, la sensibilité et la précision du modèle.